<a href="https://colab.research.google.com/github/rushi-k12/API/blob/Further_Exploration_of_Optimizations/API_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import mean_squared_error

In [2]:
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json

url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
#Original Sandbox Environment: 'https://sandbox-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
parameters = {
  'start':'1',
  'limit':'15',
  'convert':'USD'
}
headers = {
  'Accepts': 'application/json',
  'X-CMC_PRO_API_KEY': '0ad53085-1cb2-4eb8-ad9e-3ffbd7e56509',
}

session = Session()
session.headers.update(headers)

try:
  response = session.get(url, params=parameters)
  data = json.loads(response.text)
  #print(data)
except (ConnectionError, Timeout, TooManyRedirects) as e:
  print(e)

In [3]:
df = pd.json_normalize(data['data'])


In [4]:
df.columns

Index(['id', 'name', 'symbol', 'slug', 'num_market_pairs', 'date_added',
       'tags', 'max_supply', 'circulating_supply', 'total_supply',
       'infinite_supply', 'platform', 'cmc_rank',
       'self_reported_circulating_supply', 'self_reported_market_cap',
       'tvl_ratio', 'last_updated', 'quote.USD.price', 'quote.USD.volume_24h',
       'quote.USD.volume_change_24h', 'quote.USD.percent_change_1h',
       'quote.USD.percent_change_24h', 'quote.USD.percent_change_7d',
       'quote.USD.percent_change_30d', 'quote.USD.percent_change_60d',
       'quote.USD.percent_change_90d', 'quote.USD.market_cap',
       'quote.USD.market_cap_dominance', 'quote.USD.fully_diluted_market_cap',
       'quote.USD.tvl', 'quote.USD.last_updated', 'platform.id',
       'platform.name', 'platform.symbol', 'platform.slug',
       'platform.token_address'],
      dtype='object')

In [5]:


# Feature selection/engineering
# Select relevant features
# Drop the target variable from the features
features = df.drop(columns=['quote.USD.price'])
# Other feature engineering steps...

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(features, df['quote.USD.price'], test_size=0.2, random_state=42)

In [6]:
# Define the columns to keep
selected_columns = [
    'id', 'num_market_pairs', 'circulating_supply',
    'total_supply', 'infinite_supply', 'cmc_rank',
    'quote.USD.volume_24h', 'quote.USD.volume_change_24h',
    'quote.USD.percent_change_1h', 'quote.USD.percent_change_24h',
    'quote.USD.percent_change_7d', 'quote.USD.percent_change_30d',
    'quote.USD.percent_change_60d', 'quote.USD.percent_change_90d',
    'quote.USD.market_cap', 'quote.USD.market_cap_dominance',
    'quote.USD.fully_diluted_market_cap'
]

# Select only the desired columns
X_test_numeric = X_test.loc[:, selected_columns]
X_train_numeric = X_train.loc[:, selected_columns]

In [7]:
xgb_regressor = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)

In [12]:
import pandas as pd
import numpy as np
import xgboost as xgb
import joblib
from sklearn.model_selection import train_test_split

# Load your data and preprocess it
# For example:
# df = pd.read_csv('your_data.csv')
# X = df.drop(columns=['target_column'])
# y = df['target_column']

# Perform train-test split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize XGBoost regressor
model_file_path = 'xgb_regressor_model.pkl'
joblib.dump(xgb_regressor, model_file_path)


['xgb_regressor_model.pkl']

In [13]:
import pandas as pd
import numpy as np
import xgboost as xgb
import joblib
from requests import Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json

# Function to fetch new data from the API
def fetch_new_data_from_api():
    url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
    parameters = {
      'start':'1',
      'limit':'5',
      'convert':'USD'
    }
    headers = {
      'Accepts': 'application/json',
      'X-CMC_PRO_API_KEY': '0ad53085-1cb2-4eb8-ad9e-3ffbd7e56509',
    }

    session = Session()
    session.headers.update(headers)

    try:
        response = session.get(url, params=parameters)
        data = json.loads(response.text)
        return data['data']
    except (ConnectionError, Timeout, TooManyRedirects) as e:
        print(e)
        return None

# Function to preprocess new data
def preprocess_new_data(new_data):
    df = pd.json_normalize(new_data)
    df['timestamp'] = pd.to_datetime('now', utc=True)  # Fixing the FutureWarning

    # Select relevant features
    selected_columns = [
        'id', 'num_market_pairs', 'circulating_supply',
        'total_supply', 'infinite_supply', 'cmc_rank',
        'quote.USD.volume_24h', 'quote.USD.volume_change_24h',
        'quote.USD.percent_change_1h', 'quote.USD.percent_change_24h',
        'quote.USD.percent_change_7d', 'quote.USD.percent_change_30d',
        'quote.USD.percent_change_60d', 'quote.USD.percent_change_90d',
        'quote.USD.market_cap', 'quote.USD.market_cap_dominance',
        'quote.USD.fully_diluted_market_cap'
    ]
    X_test_numeric = df.loc[:, selected_columns]

    return X_test_numeric

# Function to make predictions using the loaded model
def make_predictions(model, new_data):
    preprocessed_data = preprocess_new_data(new_data)
    predictions = model.predict(preprocessed_data)
    return predictions

# Load the trained model from the file
model_file_path = 'xgb_regressor_model.pkl'
loaded_model = joblib.load(model_file_path)

# Fetch new data from the API
new_data = fetch_new_data_from_api()

# Make predictions using the loaded model
if new_data is not None:
    predictions = make_predictions(loaded_model, new_data)
    print("Predictions:", predictions)
else:
    print("Failed to fetch new data from the API.")


NotFittedError: need to call fit or load_model beforehand

In [14]:
import joblib

# Define the file path where you want to save the model
model_file_path = 'xgb_regressor_model.pkl'

# Save the trained model to the specified file path
joblib.dump(xgb_regressor, model_file_path)


['xgb_regressor_model.pkl']

In [9]:
X_train.columns

Index(['id', 'name', 'symbol', 'slug', 'num_market_pairs', 'date_added',
       'tags', 'max_supply', 'circulating_supply', 'total_supply',
       'infinite_supply', 'platform', 'cmc_rank',
       'self_reported_circulating_supply', 'self_reported_market_cap',
       'tvl_ratio', 'last_updated', 'quote.USD.volume_24h',
       'quote.USD.volume_change_24h', 'quote.USD.percent_change_1h',
       'quote.USD.percent_change_24h', 'quote.USD.percent_change_7d',
       'quote.USD.percent_change_30d', 'quote.USD.percent_change_60d',
       'quote.USD.percent_change_90d', 'quote.USD.market_cap',
       'quote.USD.market_cap_dominance', 'quote.USD.fully_diluted_market_cap',
       'quote.USD.tvl', 'quote.USD.last_updated', 'platform.id',
       'platform.name', 'platform.symbol', 'platform.slug',
       'platform.token_address'],
      dtype='object')

In [10]:
import pandas as pd
import numpy as np
import xgboost as xgb
import joblib
from requests import Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json

# Function to fetch new data from the API
def fetch_new_data_from_api():
    url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
    parameters = {
      'start':'1',
      'limit':'5',
      'convert':'USD'
    }
    headers = {
      'Accepts': 'application/json',
      'X-CMC_PRO_API_KEY': '0ad53085-1cb2-4eb8-ad9e-3ffbd7e56509',
    }

    session = Session()
    session.headers.update(headers)

    try:
        response = session.get(url, params=parameters)
        data = json.loads(response.text)
        return data['data']
    except (ConnectionError, Timeout, TooManyRedirects) as e:
        print(e)
        return None

# Function to preprocess new data
def preprocess_new_data(new_data):
    df = pd.json_normalize(new_data)
    df['timestamp'] = pd.to_datetime('now')

    # Select relevant features
    selected_columns = [
        'id', 'num_market_pairs', 'circulating_supply',
        'total_supply', 'infinite_supply', 'cmc_rank',
        'quote.USD.volume_24h', 'quote.USD.volume_change_24h',
        'quote.USD.percent_change_1h', 'quote.USD.percent_change_24h',
        'quote.USD.percent_change_7d', 'quote.USD.percent_change_30d',
        'quote.USD.percent_change_60d', 'quote.USD.percent_change_90d',
        'quote.USD.market_cap', 'quote.USD.market_cap_dominance',
        'quote.USD.fully_diluted_market_cap'
    ]
    X_test_numeric = df.loc[:, selected_columns]

    return X_test_numeric

# Function to make predictions using the loaded model
def make_predictions(model, new_data):
    preprocessed_data = preprocess_new_data(new_data)
    predictions = model.predict(preprocessed_data)
    return predictions

# Load the trained model from the .pkl file
model_file_path = 'xgb_regressor_model.pkl'
loaded_model = joblib.load(model_file_path)

# Fetch new data from the API
new_data = fetch_new_data_from_api()

# Make predictions using the loaded model
if new_data is not None:
    predictions = make_predictions(loaded_model, new_data)
    print("Predictions:", predictions)
else:
    print("Failed to fetch new data from the API.")


<ipython-input-10-eb8b9b7d3b7f>:36: FutureWarning: The parsing of 'now' in pd.to_datetime without `utc=True` is deprecated. In a future version, this will match Timestamp('now') and Timestamp.now()
  df['timestamp'] = pd.to_datetime('now')


NotFittedError: need to call fit or load_model beforehand

In [15]:
import pandas as pd
import numpy as np
import xgboost as xgb
import joblib
from requests import Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json

# Function to fetch new data from the API
def fetch_new_data_from_api():
    url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
    parameters = {
      'start':'1',
      'limit':'15',
      'convert':'USD'
    }
    headers = {
      'Accepts': 'application/json',
      'X-CMC_PRO_API_KEY': '0ad53085-1cb2-4eb8-ad9e-3ffbd7e56509',
    }

    session = Session()
    session.headers.update(headers)

    try:
        response = session.get(url, params=parameters)
        data = json.loads(response.text)
        return data['data']
    except (ConnectionError, Timeout, TooManyRedirects) as e:
        print(e)
        return None


    # Select relevant features
    selected_columns = [
        'id', 'num_market_pairs', 'circulating_supply',
        'total_supply', 'infinite_supply', 'cmc_rank',
        'quote.USD.volume_24h', 'quote.USD.volume_change_24h',
        'quote.USD.percent_change_1h', 'quote.USD.percent_change_24h',
        'quote.USD.percent_change_7d', 'quote.USD.percent_change_30d',
        'quote.USD.percent_change_60d', 'quote.USD.percent_change_90d',
        'quote.USD.market_cap', 'quote.USD.market_cap_dominance',
        'quote.USD.fully_diluted_market_cap'
    ]
    X_test_numeric = df.loc[:, selected_columns]

    return X_test_numeric

# Function to make predictions using the loaded model
def make_predictions(model, new_data):
    preprocessed_data = preprocess_new_data(new_data)
    predictions = model.predict(preprocessed_data)
    return predictions

# Load the trained model from the .pkl file
model_file_path = 'xgb_regressor_model.pkl'
loaded_model = joblib.load(model_file_path)

# Fetch new data from the API
new_data = fetch_new_data_from_api()

# Make predictions using the loaded model
if new_data is not None:
    predictions = make_predictions(loaded_model, new_data)
    print("Predictions:", predictions)
else:
    print("Failed to fetch new data from the API.")


NotFittedError: need to call fit or load_model beforehand

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
import joblib
from requests import Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json

# Function to fetch new data from the API
def fetch_new_data_from_api():
    url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
    parameters = {
      'start':'1',
      'limit':'15',
      'convert':'USD'
    }
    headers = {
      'Accepts': 'application/json',
      'X-CMC_PRO_API_KEY': '0ad53085-1cb2-4eb8-ad9e-3ffbd7e56509',
    }

    session = Session()
    session.headers.update(headers)

    try:
        response = session.get(url, params=parameters)
        data = json.loads(response.text)
        return data['data']
    except (ConnectionError, Timeout, TooManyRedirects) as e:
        print(e)
        return None


    # Select relevant features
    selected_columns = [
        'id', 'num_market_pairs', 'circulating_supply',
        'total_supply', 'infinite_supply', 'cmc_rank',
        'quote.USD.volume_24h', 'quote.USD.volume_change_24h',
        'quote.USD.percent_change_1h', 'quote.USD.percent_change_24h',
        'quote.USD.percent_change_7d', 'quote.USD.percent_change_30d',
        'quote.USD.percent_change_60d', 'quote.USD.percent_change_90d',
        'quote.USD.market_cap', 'quote.USD.market_cap_dominance',
        'quote.USD.fully_diluted_market_cap'
    ]
    X_test_numeric = df.loc[:, selected_columns]

    return X_test_numeric

# Function to make predictions using the loaded model
def make_predictions(model, new_data):
    preprocessed_data = preprocess_new_data(new_data)
    predictions = model.predict(preprocessed_data)
    return predictions

# Load the trained model from the .pkl file
model_file_path = 'xgb_regressor_model.pkl'
loaded_model = joblib.load(model_file_path)

# Fetch new data from the API
new_data = fetch_new_data_from_api()

# Make predictions using the loaded model
if new_data is not None:
    predictions = make_predictions(loaded_model, new_data)
    print("Predictions:", predictions)
else:
    print("Failed to fetch new data from the API.")


<ipython-input-26-eb8b9b7d3b7f>:36: FutureWarning: The parsing of 'now' in pd.to_datetime without `utc=True` is deprecated. In a future version, this will match Timestamp('now') and Timestamp.now()
  df['timestamp'] = pd.to_datetime('now')


NotFittedError: need to call fit or load_model beforehand

In [11]:
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json
import pandas as pd
import numpy as np
import xgboost as xgb
import joblib
from sklearn.model_selection import train_test_split

# Define the function to fetch new data from the API
def fetch_new_data_from_api():
    url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
    parameters = {
      'start':'1',
      'limit':'5',
      'convert':'USD'
    }
    headers = {
      'Accepts': 'application/json',
      'X-CMC_PRO_API_KEY': '0ad53085-1cb2-4eb8-ad9e-3ffbd7e56509',
    }

    session = Session()
    session.headers.update(headers)

    try:
        response = session.get(url, params=parameters)
        data = json.loads(response.text)
        return data['data']
    except (ConnectionError, Timeout, TooManyRedirects) as e:
        print(e)
        return None

# Define the function to preprocess new data
def preprocess_new_data(new_data):
    df = pd.json_normalize(new_data)
    df['timestamp'] = pd.to_datetime('now')

    # Select relevant features
    selected_columns = [
        'id', 'num_market_pairs', 'circulating_supply',
        'total_supply', 'infinite_supply', 'cmc_rank',
        'quote.USD.volume_24h', 'quote.USD.volume_change_24h',
        'quote.USD.percent_change_1h', 'quote.USD.percent_change_24h',
        'quote.USD.percent_change_7d', 'quote.USD.percent_change_30d',
        'quote.USD.percent_change_60d', 'quote.USD.percent_change_90d',
        'quote.USD.market_cap', 'quote.USD.market_cap_dominance',
        'quote.USD.fully_diluted_market_cap'
    ]
    X_test_numeric = df.loc[:, selected_columns]

    return X_test_numeric

# Define the function to make predictions using the loaded model
def make_predictions(model, new_data):
    preprocessed_data = preprocess_new_data(new_data)
    predictions = model.predict(preprocessed_data)
    return predictions

# Load the trained model from the .pkl file
model_file_path = 'xgb_regressor_model.pkl'
loaded_model = joblib.load(model_file_path)

# Fetch new data from the API
new_data = fetch_new_data_from_api()

# Make predictions using the loaded model
if new_data is not None:
    predictions = make_predictions(loaded_model, new_data)
    print("Predictions:", predictions)
else:
    print("Failed to fetch new data from the API.")



<ipython-input-11-47c5485c472d>:37: FutureWarning: The parsing of 'now' in pd.to_datetime without `utc=True` is deprecated. In a future version, this will match Timestamp('now') and Timestamp.now()
  df['timestamp'] = pd.to_datetime('now')


NotFittedError: need to call fit or load_model beforehand

In [22]:
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json
import pandas as pd
import numpy as np
import xgboost as xgb
import joblib
from sklearn.model_selection import train_test_split

# Function to fetch new data from the API
def fetch_new_data_from_api():
    url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
    parameters = {
      'start':'1',
      'limit':'5',
      'convert':'USD'
    }
    headers = {
      'Accepts': 'application/json',
      'X-CMC_PRO_API_KEY': '0ad53085-1cb2-4eb8-ad9e-3ffbd7e56509',
    }

    session = Session()
    session.headers.update(headers)

    try:
        response = session.get(url, params=parameters)
        data = json.loads(response.text)
        return data['data']
    except (ConnectionError, Timeout, TooManyRedirects) as e:
        print(e)
        return None

# Function to preprocess new data
# Function to preprocess new data
def preprocess_new_data(data):
    df = pd.json_normalize(data)
    df['timestamp'] = pd.to_datetime('now', utc=True)  # Fixing the FutureWarning

    # Select relevant features
    selected_columns = [
        'id', 'num_market_pairs', 'circulating_supply',
        'total_supply', 'infinite_supply', 'cmc_rank',
        'quote.USD.volume_24h', 'quote.USD.volume_change_24h',
        'quote.USD.percent_change_1h', 'quote.USD.percent_change_24h',
        'quote.USD.percent_change_7d', 'quote.USD.percent_change_30d',
        'quote.USD.percent_change_60d', 'quote.USD.percent_change_90d',
        'quote.USD.market_cap', 'quote.USD.market_cap_dominance',
        'quote.USD.fully_diluted_market_cap'
    ]
    X_test_numeric = df.loc[:, selected_columns]

    return X_test_numeric

# Function to make predictions using the loaded model
def make_predictions(model, data):
    preprocessed_data = preprocess_new_data(new_data)
    predictions = model.predict(preprocessed_data)
    return predictions

# Train-test split
# Replace this with your actual data loading and preprocessing steps
# X_train, X_test, y_train, y_test = train_test_split(features, df['quote.USD.price'], test_size=0.2, random_state=42)

# Assuming you have already trained your model
# Replace this with your actual model training code
xgb_regressor = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)
# xgb_regressor.fit(X_train, y_train)  # Train your model

# Save the trained model to a file
model_file_path = 'xgb_regressor_model.pkl'
joblib.dump(xgb_regressor, model_file_path)

# Load the trained model from the file
loaded_model = joblib.load(model_file_path)

# Fetch new data from the API
data = fetch_new_data_from_api()

# Make predictions using the loaded model
if data is not None:
    predictions = make_predictions(loaded_model, data)
    print("Predictions:", predictions)
else:
    print("Failed to fetch new data from the API.")



NotFittedError: need to call fit or load_model beforehand